# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_file = os.path.join('..', 'output_data', 'cities.csv')

city_df = pd.read_csv(csv_file)
city_df

,Unnamed: 0,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,brits,-25.6347,27.7802,293.80,55,67,2.29,ZA,1648965817
1,1,bethel,41.3712,-73.4140,277.56,77,0,1.34,US,1648965817
2,2,bathsheba,13.2167,-59.5167,298.12,72,39,8.83,BB,1648965818
3,3,lake city,30.1897,-82.6393,287.48,100,100,0.00,US,1648965820
4,4,punta arenas,-53.1500,-70.9167,280.99,70,56,10.78,CL,1648965820
...,...,...,...,...,...,...,...,...,...,...
550,550,jarva-jaani,59.0386,25.8864,270.53,79,26,2.89,EE,1648966067
551,551,qasigiannguit,68.8193,-51.1922,271.29,91,88,3.86,GL,1648966067
552,552,sandanski,41.5667,23.2833,280.29,83,91,2.11,BG,1648966067
553,553,sirjan,29.4520,55.6814,296.07,8,0,3.38,IR,1648966068


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_df['MaxTemp'] = (city_df['MaxTemp'] - 273.15)*1.8 + 32
ideal_df = city_df[(city_df['MaxTemp'] >= 70) & (city_df['MaxTemp'] <= 80)]
ideal_df = ideal_df[ideal_df['Wind Speed'] <= 10]
ideal_df = ideal_df[ideal_df['Cloudiness'] == 0]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
37,37,tsaratanana,-16.7833,47.6500,79.070,62,0,2.50,MG,1648965837,
182,182,ambovombe,-25.1667,46.0833,77.252,61,0,2.01,MG,1648965902,
278,278,yining,29.0361,114.5636,70.718,32,0,2.02,CN,1648965945,
350,350,abha,18.2164,42.5053,71.852,9,0,2.06,SA,1648965978,
384,384,guilin,25.2819,110.2864,72.428,33,0,4.57,CN,1648965771,
423,423,ginda,30.6945,78.4932,79.592,8,0,3.77,IN,1648966010,
426,426,makakilo city,21.3469,-158.0858,73.724,72,0,9.77,US,1648965995,
432,432,joshimath,30.5667,79.5667,70.610,26,0,3.05,IN,1648966014,
537,537,ewa beach,21.3156,-158.0072,75.146,72,0,9.77,US,1648966062,
553,553,sirjan,29.4520,55.6814,73.256,8,0,3.38,IR,1648966068,


In [7]:
for index, row in hotel_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }
        
        lat = row['Lat']
        lng = row['Lng']
        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df        

,Unnamed: 0,City,Lat,Lng,MaxTemp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
37,37,tsaratanana,-16.7833,47.6500,79.070,62,0,2.50,MG,1648965837,NaN
182,182,ambovombe,-25.1667,46.0833,77.252,61,0,2.01,MG,1648965902,Hotel Le Source
278,278,yining,29.0361,114.5636,70.718,32,0,2.02,CN,1648965945,Radisson Hotel
350,350,abha,18.2164,42.5053,71.852,9,0,2.06,SA,1648965978,OYO 315 Ramz Abha Hotel
384,384,guilin,25.2819,110.2864,72.428,33,0,4.57,CN,1648965771,guilin west street hotel
423,423,ginda,30.6945,78.4932,79.592,8,0,3.77,IN,1648966010,Hotel Ashirwad
426,426,makakilo city,21.3469,-158.0858,73.724,72,0,9.77,US,1648965995,Hampton Inn & Suites Oahu/Kapolei
432,432,joshimath,30.5667,79.5667,70.610,26,0,3.05,IN,1648966014,Hotel Auli D - Hotel In Joshimath
537,537,ewa beach,21.3156,-158.0072,75.146,72,0,9.77,US,1648966062,NaN
553,553,sirjan,29.4520,55.6814,73.256,8,0,3.38,IR,1648966068,هتل سیرجان


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

fig.add_layer(markers)
fig.add_layer(hotel_layer)

# Display figure
fig